# Using Llama-3

Choosing the GPU (if you use CUDA, the command nvidia-smi will  tell you the status of all your CPUs)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Importing tokenizer and model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
)

Creating the message, with multiple roles for the system (llama3) and the user's prompt

In [ ]:
messages = [
    {"role": "system", "content": "You are a car expert"},
    {"role": "user", "content": "Tell me about Ferrari cars"},
]

Tokenizing the message

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

Setting llama3 terminators

In [ ]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Generation (need to specify the end of string tokens)

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

What if we print the first response?

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The response contains the prompt, so we remove the first N tokens counting those of the input

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))